### SQL Server

### Index

In sql server, index is a database object that improves the speed of data retrieval oprations on a table or view.It creates a structure that allows SQL Server to locate rows more efficiently. An index is used to quickly look up data based on the values of one or more columns, significantly speeding up query performance, especially for large datasets.

- Advantages:
1. Speeds up the query: `Select, Join, Group by, Order by`

2. Cheas for the connection of foreign key

3. Clustering: changes phisical order according to the index

4. Filtering becomes faster as in a huge tables an exact line will be found in seconds

5. Segment elementation: in columnstore indexes, necessery segmentation will be readen

### Index Types

- Clustered Indexing

`Clustered Index` in SQL server determines physical order of data rowas in a table: Table can have only one index of this type

In [ ]:
create clustered index ix_id on MyTable_id

Why do we need this: query beomes faster as there is no need to read each row

In [ ]:
select * from MyTable
where id between 2 and 5

--------------------------------------------------------------------------------

- Non Clustered

`Non clustered index` is an index that createsa a seperate structure from the actual data table. It contains a poineter to the table rows rather than storing the data itself. A table can have multiple non-clustered indexes, and they improve query performance by allowing faster lookups based on indexed columns

In [ ]:
CREATE NONCLUSTERED INDEX IX_Name ON MyTable(Name);

--------------------------------------------------------------------------------

- Key Lookup

A key lookup in SQL Server occurs when a query requires additional data that is not available in the non-clustered index, so SQL Server must perform a lookup in the clustered index (or table) to retrieve the missing information. This happens when the query includes columns that are not part of the non-clustered index, causing SQL Server to fetch the data from the table using the pointer stored in the non-clustered index. Key lookups can impact performance, especially if the lookup operation is frequent or involves large amounts of data.

`Include`: is used to avoid key look up as they really slow down the speed

In [ ]:
CREATE NONCLUSTERED INDEX IX_Name_Email ON Users(Name)
INCLUDE (Email);

When name column got an index but the other one no, through the index in name column, though all the rows and columns one by one will be checked in order to get the values where index lucks

Index Disadvantages

1. Takes a lot of place in a disk

2. `Insert, Update` will be slowed down

3. Requires a regular care: `Rebuil, Reorgonize`

--------------------------------------------------------------------------------

### Index = seperate file

Each index is a data scructure in a file, indexes are keeped in fileas .mdf: index is a structure in a file in simple words

In SQL Server all information: rows, tables, views, indexes are keept in a pages(.mdf)

- List of all indexes

In [ ]:
SELECT * FROM sys.indexes WHERE object_id = OBJECT_ID('Users');

--------------------------------------------------------------------------------

### B+Tree

A B+ tree is a type of data structure commonly used for indexing in databases, including SQL Server

The B+ tree allows for fast searching, insertion, and deletion of records by efficiently navigating the tree using the pointers. It's especially effective for range queries (like finding all records within a certain range) because the leaf nodes are linked, allowing for easy sequential access.

In SQL Server, both clustered and non-clustered indexes are typically implemented using B+ trees.


             Root
            /    \
         Node    Node
        /   \    /   \
      Page Page Page Page



--------------------------------------------------------------------------------

### Index Types

- Unique Index

Provides with uniques values

In [ ]:
CREATE UNIQUE INDEX IX_Email ON Users(Email);

- Filtered Index

Results of an exact cond will get an index: City = 'New York'

- Composite Index

Several rows will be indexed together

- Included Columns

In SQL Server, an **included column** in a non-clustered index is a column that is added to the index for **performance reasons** but is not part of the key of the index. These columns are stored in the index itself, so when a query requests them, SQL Server can retrieve the data directly from the index without needing to access the table. This can improve query performance by reducing the need for lookups.

- Columnstore Index

A Columnstore index in SQL Server is a type of index that stores data in a columnar format rather than the traditional row-based format. This allows for efficient data compression and faster query performance, especially for large data sets or analytical queries. Columnstore indexes are ideal for read-heavy operations, like data warehousing and OLAP (Online Analytical Processing) workloads, where queries typically involve scanning large volumes of data.

- XML Index

An XML index in SQL Server is a specialized index used to improve the performance of queries that retrieve or modify XML data stored in an XML column.

- Spatial Index

Works with geraphical or geometrical cordinates

- Full-Text Index

Works with txt columns: speeds up `CONTAINS, FREETEXT`

In [ ]:
# Clustered (changes table order)
CREATE CLUSTERED INDEX idx_cust_id ON Customers(CustomerID);

# Non-clustered
CREATE NONCLUSTERED INDEX idx_cust_name ON Customers(CustomerName);

# Unique (no duplicate emails!)
CREATE UNIQUE INDEX idx_email ON Customers(Email);

# Composite (name + birthdate together)
CREATE INDEX idx_name_dob ON Customers(FirstName, BirthDate);

# Filtered (only active customers)
CREATE NONCLUSTERED INDEX idx_active ON Customers(Status) WHERE Status = 'Active';

# Columnstore (for big data tables)
CREATE CLUSTERED COLUMNSTORE INDEX idx_sales ON SalesData;

### Optimizer Index

- There is no special index type called `optimizer index`

- SQL Server Query Optimizer is the brain that decides which index to use to reach a better results

- Optimizer Index = not a real index type — it's just the index chosen by the query optimizer during query execution.


How Optimizer index works: `Seek, Scan, Key Look up, Index Intersection, SARGable`

`Index Seek`: Process of getting at once the right value using indexes

"Go straight to the shelf where the book is — not checking every book one by one."

In [ ]:
SELECT * FROM Users WHERE Id = 5;

--------------------------------------------------------------------------------

`Index Scan`: Reades the entire index from top to bottom

              - Query cannot directly jump to data.
              - Many rows match.
              - No good WHERE condition.

`Table Scan`: Reads entire table as no index can help

    - Index Seek	🎯 Jumps directly to data (Best)
    - Index Scan	📄 Reads all index entries
    - Table Scan	🏛️ Reads all table rows

In [ ]:
SELECT * FROM Users WHERE Email LIKE '%mail.com';

--------------------------------------------------------------------------------

`Index Intersection`: Index Intersection happens when SQL Server uses two or more indexes together to answer a query instead of just one.


           - SQL Server uses idx_first_name to find all employees named 'John'.
           - SQL Server uses idx_last_name to find all employees with last name 'Doe'.
           - Then it intersects (finds common emp_id) from both results = Index
             Intersection.

In [ ]:
SELECT * FROM Users WHERE Name = 'Bob' AND Age = 30;

--------------------------------------------------------------------------------

`Index Union`: Index Union happens when SQL Server uses multiple indexes to find rows separately, and then combines (UNIONs) the results together.


            - One condition uses first_name.
            - One condition uses age.
            - Searches first_name index for 'John'.
            - Searches age index for 30.
            - Then combines (UNIONs) all results = Index Uni

In [ ]:
SELECT * FROM Users WHERE Name = 'Bob' OR Age = 30;

--------------------------------------------------------------------------------

`SARGable(Serach Argument able)`:

      - 👉 It means SQL Server (or any database engine) can use an index efficiently while executing your query.

      - If your WHERE, JOIN, or HAVING condition is SARGable, SQL can seek directly into indexes → FAST 🚀.

      - If it’s NOT SARGable, SQL must scan rows → SLOW 🐢.

In [ ]:
# Examples: SARGable
SELECT * FROM Users WHERE Age = 30;

SELECT * FROM Users WHERE Name LIKE 'A%';

SELECT * FROM Orders WHERE OrderDate BETWEEN '2023-01-01' AND '2023-12-31';

--------------------------------------------------------------------------------

`Non-SARGable` examples

In [ ]:
# YEAR(BirthDate)vfunc changing the column
SELECT * FROM Users WHERE YEAR(BirthDate) = 2020;

# starting with %,  index is useless
SELECT * FROM Users WHERE Name LIKE '%bob%';

# Arithmetics
SELECT * FROM Orders WHERE OrderDate + 1 = '2023-01-01';

# ISNULL, UPPER, LOWER
SELECT * FROM Users WHERE ISNULL(Email, '') = 'example@mail.com';